In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def generate_signal(df):
    #Write the signal here
    df['signal']=[1]*len(df)
    # df['signal'].iloc[::10000]=0
    
    #Set the last signal to 0 so that we close position at the end of the day
    df['signal'].iloc[-1]=0
    return df

In [ ]:
import os

# Directory containing the CSV files
directory = 'equity-data'
bbo_cols=['bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_ct_00','ask_ct_00', 'symbol']
cumulative_returns = pd.Series()
# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory, filename)
        print(f"Processing {filename}")
        df = pd.read_csv(filepath, parse_dates=['ts_recv', 'ts_event'])
        print(f"Processed {filename}")
        # Perform any additional processing on df here
        nvda_bbo=d1[d1['symbol']=='NVDA'].set_index('ts_event').resample('100ms').last().ffill()
        nvda_bbo['signal']=generate_signal(nvda_bbo)
        nvda_bbo['return']=0

        #backtesting
        zero_indices = nvda_bbo[(nvda_bbo['signal'] == 0) & (nvda_bbo['signal'].shift(1).fillna(0)!= 0)].index

        for zero_index in tqdm(zero_indices):
        # Find the last zero index behind the set of non-zero values
            last_zero_index = nvda_bbo.loc[:zero_index-pd.Timedelta(milliseconds=100)][nvda_bbo['signal'] == 0].index[-1]
            #print(last_zero_index)
            first_buy_index = last_zero_index + pd.Timedelta(milliseconds=100)
            #print(first_buy_index)
            if first_buy_index in nvda_bbo.index:
                first_buy_price = nvda_bbo.loc[first_buy_index, 'ask_px_00']
                sell_price = nvda_bbo.loc[zero_index, 'bid_px_00']
                nvda_bbo.loc[zero_index, 'return'] = ((sell_price - first_buy_price) / first_buy_price)

        nvda_bbo['cumulative_return'] = (nvda_bbo['return']).cumsum()
        cumulative_returns=pd.concat([cumulative_returns,nvda_bbo['cumulative_return']])
        